## Data Analysis Portion for Prospective Los Angeles Neighborhoods

Import needed libraries 

In [32]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import random # library for random number generation

!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

! pip install folium==0.5.0
import folium # map rendering library

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### This section will be to create dataframe and map of Los Angeles neighborhood data

Scrap the wiki table from http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_LA_Neighborhoods_Data to get Los Angeles Neighborhood Data that includes Latitude and Longitude of each LA neighborhood

In [34]:
from bs4 import BeautifulSoup

wiki_url = 'http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_LA_Neighborhoods_Data'
req = requests.get(wiki_url)

soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[2]
df = pd.read_html(str(table))
df = pd.DataFrame(df[0])
df

,LA_Nbhd,Income,Schools,Diversity,Age,Homes,Vets,Asian,Black,Latino,White,Population,Area,Longitude,Latitude
0,Adams_Normandie,29606,691,0.6,26,0.26,0.05,0.05,0.25,0.62,0.06,31068,0.8,-118.300270,34.030970
1,Arleta,65649,719,0.4,29,0.29,0.07,0.11,0.02,0.72,0.13,31068,3.1,-118.430015,34.240603
2,Arlington_Heights,31423,687,0.8,31,0.31,0.05,0.13,0.25,0.57,0.05,22106,1.0,-118.320109,34.043611
3,Atwater_Village,53872,762,0.9,34,0.34,0.06,0.20,0.01,0.51,0.22,14888,1.8,-118.265808,34.124908
4,Baldwin_Hills/Crenshaw,37948,656,0.4,36,0.36,0.10,0.05,0.71,0.17,0.03,30123,3.0,-118.366700,34.019090
5,Bel-Air,208861,924,0.2,46,0.46,0.13,0.08,0.01,0.05,0.83,7928,6.6,-118.463558,34.096148
6,Beverly_Crest,168104,0,0.1,45,0.45,0.10,0.04,0.02,0.03,0.88,10610,7.9,-118.424713,34.112107
7,Beverly_Grove,63039,791,0.2,38,0.38,0.05,0.05,0.02,0.06,0.82,21417,1.7,-118.372111,34.076009
8,Beverlywood,105253,872,0.2,39,0.39,0.08,0.07,0.04,0.06,0.80,6080,0.8,-118.395011,34.041610
9,Boyle_Heights,33235,689,0.1,25,0.25,0.03,0.02,0.01,0.94,0.02,92785,6.5,-118.205370,34.033700


Rename columns LA_Nbhd to Neighborhoods

In [36]:
la_neighborhoods = df.rename(columns={"LA_Nbhd": "Neighborhood"})

Only want Neighborhood, Longitude and Latitude. Drop all other columns.
Also, rename df to la_neighborhoods

In [37]:
la_neighborhoods.drop(['Income','Schools','Diversity','Age','Homes','Vets','Asian','Black','Latino','White','Population','Area'], axis=1, inplace=True)
la_neighborhoods.head()

,Neighborhood,Longitude,Latitude
0,Adams_Normandie,-118.300270,34.030970
1,Arleta,-118.430015,34.240603
2,Arlington_Heights,-118.320109,34.043611
3,Atwater_Village,-118.265808,34.124908
4,Baldwin_Hills/Crenshaw,-118.366700,34.019090


Use geopy to get Latitude and Longitude of Los Angeles

In [30]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.2427666.


Create a map of Los Angeles with the neighborhoods superimposed on top

In [57]:
# create map of Manhattan using latitude and longitude values
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(la_neighborhoods['Latitude'], la_neighborhoods['Longitude'], la_neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
    
map_la

### This next section is to make calls to the Foursquare API to get the common venues in each neighborhood, and find the neighborhoods with Coffee Shop as one of their top venues

Foursquare Credentials (will hide this cell as it contains sensitive data)

In [46]:
CLIENT_ID = 'LEUBN1WC4ZQQKJ0THV0RRI1CIOMTMGRETOTNNDDHGKJLAQ5L' # Foursquare ID
CLIENT_SECRET = 'PCFPI2NX1I0NAABERX5CBPSHWTBLZ2IXITA22PYVL4QK1L2R' #Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LEUBN1WC4ZQQKJ0THV0RRI1CIOMTMGRETOTNNDDHGKJLAQ5L
CLIENT_SECRET:PCFPI2NX1I0NAABERX5CBPSHWTBLZ2IXITA22PYVL4QK1L2R


Get the venue category of each venue (same as in lab)

In [47]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Create function to get venues in all Los Angeles neighborhoods listed in above dataframe

In [48]:
LIMIT = 100 #limit to 100 venues per neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Use above function to create dataframe that has nearby venues for each neighborhood in Los Angeles

In [49]:
la_venues = getNearbyVenues(names=la_neighborhoods['Neighborhood'],
                                  latitudes=la_neighborhoods['Latitude'],
                                  longitudes=la_neighborhoods['Longitude'])

Adams_Normandie
Arleta
Arlington_Heights
Atwater_Village
Baldwin_Hills/Crenshaw
Bel-Air
Beverly_Crest
Beverly_Grove
Beverlywood
Boyle_Heights
Brentwood
Broadway_Manchester
Canoga_Park
Carthay
Central_Alameda
Century_City
Chatsworth
Chesterfield_Square
Cheviot_Hills
Chinatown
Cypress_Park
Del_Rey
Downtown
Eagle_Rock
East_Hollywood
Echo_Park
El_Sereno
Elysian_Park
Elysian_Valley
Encino
Exposition_Park
Fairfax
Florence
Glassell_Park
Gramercy_Park
Granada_Hills
Green_Meadows
Hancock_Park
Harbor_City
Harbor_Gateway
Harvard_Heights
Harvard_Park
Highland_Park
Historic_South_Central
Hollywood
Hollywood_Hills
Hollywood_Hills_West
Hyde_Park
Jefferson_Park
Koreatown
Lake_Balboa
Lake_View_Terrace
Larchmont
Leimert_Park
Lincoln_Heights
Los_Feliz
Manchester_Square
Mar_Vista
Mid_City
Mid_Wilshire
Mission_Hills
Montecito_Heights
Mount_Washington
North_Hills
North_Hollywood
Northridge
Pacific_Palisades
Pacoima
Palms
Panorama_City
Pico_Robertson
Pico_Union
Playa_del_Rey
Playa_Vista
Porter_Ranch
Rancho_P

In [50]:
la_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adams_Normandie,34.03097,-118.30027,Orange Door Sushi,34.032485,-118.299368,Sushi Restaurant
1,Adams_Normandie,34.03097,-118.30027,Shell,34.033095,-118.300025,Gas Station
2,Adams_Normandie,34.03097,-118.30027,Little Xian,34.032292,-118.299465,Sushi Restaurant
3,Adams_Normandie,34.03097,-118.30027,Sushi Delight,34.032501,-118.299454,Sushi Restaurant
4,Adams_Normandie,34.03097,-118.30027,Tacos La Estrella,34.032230,-118.300757,Taco Place


Analyze the neighborhoods

In [51]:
# one hot encoding
la_onehot = pd.get_dummies(la_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
la_onehot['Neighborhood'] = la_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]

la_onehot.head()

,Yoga Studio,ATM,Accessories Store,Adult Boutique,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Trail,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Classroom,College Quad,College Residence Hall,College Theater,Comedy Club,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Romanian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,School,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Trail,Train Station,Transportation Service,Tunnel,Udon Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Watch Shop,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Group the neighborhoods and calculate the frequency of type of venue per neighborhood

In [52]:
la_grouped = la_onehot.groupby('Neighborhood').mean().reset_index()
la_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Trail,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Classroom,College Quad,College Residence Hall,College Theater,Comedy Club,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Music Venue,Nail Salon,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Romanian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,School,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Trail,Train Station,Transportation Service,Tunnel,Udon Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Watch Shop,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adams_Normandie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

Create function to sort venue frequency in descending order

In [53]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create dataframe to show the top 5 most frequent venue type per neighborhood

In [55]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = la_grouped['Neighborhood']

for ind in np.arange(la_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(la_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Adams_Normandie,Sushi Restaurant,Playground,Grocery Store,Gas Station,Taco Place
1,Arleta,Video Store,Historic Site,Bakery,Home Service,Convenience Store
2,Arlington_Heights,Rental Car Location,Restaurant,Café,Shop & Service,Donut Shop
3,Atwater_Village,Chinese Restaurant,Food Truck,Mobile Phone Shop,Fast Food Restaurant,Bank
4,Baldwin_Hills/Crenshaw,Flower Shop,Clothing Store,Women's Store,Ethiopian Restaurant,Food Truck


Finally, create new dataframe with only the Los Angeles neighborhoods that have "Coffee Shop" as their #1 most common venue

In [56]:
neighborhoods_venues_sorted_coffeeshop = neighborhoods_venues_sorted[neighborhoods_venues_sorted['1st Most Common Venue'] == "Coffee Shop"].reset_index(drop=True)
neighborhoods_venues_sorted_coffeeshop

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Canoga_Park,Coffee Shop,Thrift / Vintage Store,Rental Car Location,Asian Restaurant,Pharmacy
1,Los_Feliz,Coffee Shop,Mexican Restaurant,Trail,Grocery Store,Bank
2,Mid_City,Coffee Shop,Pharmacy,Rock Club,Farmers Market,Filipino Restaurant
3,Mid_Wilshire,Coffee Shop,Cosmetics Shop,French Restaurant,Burger Joint,Sake Bar
4,Palms,Coffee Shop,Mediterranean Restaurant,Convenience Store,Thai Restaurant,Gym
5,Porter_Ranch,Coffee Shop,Department Store,Fast Food Restaurant,Gift Shop,Clothing Store
6,Silver_Lake,Coffee Shop,Thai Restaurant,Bakery,Accessories Store,Salon / Barbershop
7,Studio_City,Coffee Shop,Italian Restaurant,Gift Shop,Sandwich Place,Park
8,University_Park,Coffee Shop,Pizza Place,American Restaurant,Food Truck,Restaurant
